In [1]:
%load_ext autoreload
%autoreload 2
from AmbientesMP import *
from random import choice
from busqueda import Nodo , nodo_hijo

## BREAD FIRST SEARCH

In [2]:
def breadth_first_search(prob):
    
    node = Nodo(estado = prob.estado_inicial,
                madre  = None, 
                accion = None, 
                costo_camino = 0, 
                codigo = "origin")
    
    if prob.test_objetivo(node.estado):
        return node
    
    frontier = [node]
    explored = []  
    
        
    while len(frontier) > 0:
        node = frontier.pop(0)
        explored.append(node.codigo)
        
        for action in prob.acciones_aplicables(node.estado):
            child = nodo_hijo(prob, node, action)
            
            if prob.test_objetivo(child.estado):
                return child
            if child.codigo not in explored:
                frontier.append(child)

            
    return False

## DEPTH LIMITED SEARCH

In [3]:
def get_codes(node: Nodo):
    
    if node.madre == None:
        return [node.codigo]
    
    return  [] + [node.codigo] + get_codes(node.madre)

In [4]:
def is_cycle(node: Nodo):
    
    return len( set(get_codes(node)) ) != len(get_codes(node))

In [5]:
def expand(problema, node):
    
    state = node.estado
    nodes = []
    
    for action in problema.acciones_aplicables(state):
        child = nodo_hijo(problema, node, action)
        nodes.append(child)
        
    return nodes

In [6]:
def depth_(node):
    
    if node.madre == None:
        return 0
    
    return 1 + depth_(node.madre) 

In [7]:
def depth_limited_search(prob, l):
    
    node = Nodo(estado = prob.estado_inicial,
                madre  = None,
                accion = None, 
                costo_camino = 0,
                codigo = 'origin')
    
    frontier  = [node]
    result = "falla"
    
    while len(frontier) > 0:
        
        node = frontier.pop()
        
        if prob.test_objetivo(node.estado):
            return node
        
        if depth_(node) >= l:
            continue
            #return "cutoff"
        
        elif not is_cycle(node):
            
            for child in expand(prob, node):
                frontier.append(child)
                
    return result

## BEST FIRST SEARCH

In [8]:
import numpy as np

class ListaPrioritaria():
    
    def __init__(self):
        self.diccionario = {}
        
    def __str__(self):
        cadena = '['
        inicial = True
        for costo in self.diccionario:
            elementos = self.diccionario[costo]
            for elemento in elementos:
                if inicial:
                    cadena += '(' + str(elemento) + ',' + str(costo) + ')'
                    inicial = False
                else:
                    cadena += ', (' + str(elemento) + ',' + str(costo) + ')'

        return cadena + ']'
    
    def push(self, elemento, costo):
        try:
            self.diccionario[costo].append(elemento)
        except:
            self.diccionario[costo] = [elemento]
            
    def pop(self):
        min_costo  = min(list(self.diccionario.keys()))
        candidatos = self.diccionario[min_costo]
        elemento   = candidatos.pop()
        
        if len(candidatos) == 0:
            del self.diccionario[min_costo]
        return elemento
    
    def is_empty(self):
        return len(self.diccionario) == 0

In [9]:
def expand(problema, node):
    
    state = node.estado
    nodes = []
    
    for action in problema.acciones_aplicables(state):
        child = nodo_hijo(problema, node, action)
        nodes.append(child)
        
    return nodes

In [10]:
def costs(self, state, action):
    
    """
        INPUT:
                action: tupla de la forma ('S', 'aSb'), por ejemplo.
                
        OUTPUT:
                costo de tomar la acción (dependerá de que tan largo 
                sea el string, y esto lo hacemos para incentivar
                que acabe rápido)
    """

    return len(action[1])

In [11]:
from types import MethodType

def best_first_search(problem, fun):
    
    #asigna la función "fun" como función de costo
    problem.costo = MethodType(fun, problem)
    
    state = problem.estado_inicial
    code  = problem.codigo(state)
    node  = Nodo(estado = state,
                 madre  = None,
                 accion = None, 
                 costo_camino = 0,
                 codigo = 'origin')
    
    # pongo primer elemento en la frontera
    frontier = ListaPrioritaria()
    frontier.push(elemento = node,
                  costo    = 0)
    
    #explored dict
    explored = {code: 0}
    
    while not frontier.is_empty():
        
        node = frontier.pop()
        
        if problem.test_objetivo(node.estado):
            return node
    
        for child in expand(problem, node):
                
            state = child.estado
            code  = problem.codigo(state)
            
            if len(child.codigo) > len(problem.w):
                continue
                
            cost  = child.costo_camino
            
            if (code not in explored.keys()) or (cost < explored[code]):
                frontier.push(child, cost)
                explored[code]  = cost
                
    return "falla"

## GREEDY SEARCH

In [12]:
def error_function(state: str, obj: str) -> int:
    
    """
        Input:
            state: String de la forma "aaSbb" o "aabb" por ejemplo.
            
        Output:
            nuestra loca euristica (int)
    """

    if len(obj) == len(state):
        char_distance = sum([ 1 for l1, l2 in zip(list(obj), list(state))  if l1 != l2])**2
    
    else:
        char_distance = len(obj)**2
        
    lenght_distance = ( (len(obj) - len(state))**2 )

    return lenght_distance + char_distance

In [13]:
def greedy_search(problema, f):
    
    state = problema.estado_inicial
    node  = Nodo(estado = state, 
                 madre  = None, 
                 accion = None,
                 costo_camino = 0, 
                 codigo = "orgin")
    
    cost = f(problema.codigo(state), problema.w)
    
    frontier = ListaPrioritaria()
    frontier.push(node, cost)
    
    code = problema.codigo(state)
    explored = [code]
    
    while not frontier.is_empty():
        node = frontier.pop()

        if problema.test_objetivo(node.estado):
            return node
        
        for child in expand(problema, node):

            state = child.estado
            code  = problema.codigo(state)
            
            if len(code) > len(problema.w):
                continue

            if code not in explored:
                cost  = f(problema.codigo(state), problema.w)
                frontier.push(child, cost)
                explored.append(code)
    
    return None

## PRUEBAS

In [14]:
%load_ext autoreload
%autoreload 2
from example_grammars import ExampleGrammars

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
e = ExampleGrammars()

grammar_n = 4
gramatica = e.get_grammar(grammar_num = grammar_n)

w = e.generate_word(grammar_num = grammar_n,
                    word_interval_len = (50, 100))
prob = Parser(gramatica, w)
print(w, len(w))

aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaabbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbb 80


In [16]:
ans = best_first_search(problem = prob, fun = costs)
ans

In [17]:
prob.pintar_estado(ans.estado)

                                                                                                                                                             S                                                                                                                                                                     
  ___________________________________________________________________________________________________________________________________________________________|___                                                                                                                                                                   
 |   |                                                                                                                                                           S                                                                                                                                                                 
 |   |    _________________

In [18]:
ans = breadth_first_search(prob  = prob)
ans

In [19]:
ans = depth_limited_search(prob = prob, l = len(w)*1)
ans

In [20]:
ans = greedy_search(prob, error_function)
ans

## GRÁFICAS

In [22]:
%load_ext autoreload
%autoreload 2
from example_grammars import ExampleGrammars
from tiempos import *
from random import sample
import warnings
warnings.filterwarnings('ignore')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
e = ExampleGrammars()

funs = [
        lambda x: best_first_search(x, costs),
        lambda x: greedy_search(x, error_function),
        lambda x: depth_limited_search(x, len(x.w)),
        lambda x: breadth_first_search(x)
       ]

nombres = [
            'Best friend', 
            'Greedy friend',
            'Depth search',
            'Breadth search'
          ]

word_interval_len = (50, 100)
n_times = 5
args = []

for i in range(n_times):

    grammar_n = choice((1, 2, 3, 4))
    
    grammar = e.get_grammar(grammar_num = grammar_n)
    w = e.generate_word(grammar_num = grammar_n,
                        word_interval_len = word_interval_len)
    
    args.append(Parser(grammar, w))

data = compara_funciones_sobreargumentos(funs=funs, args=args, nombres=nombres)

# Graficando
fig, ax = plt.subplots(1,1, figsize=(3*len(funs),3), tight_layout=True)
sns.boxplot(data=data, x='Función', y='Tiempo')
sns.swarmplot(data=data, x='Función', y='Tiempo', color='black', alpha = 0.5, ax=ax);

# Anova diferencia de medias
model = ols('Tiempo ~ C(Función)', data=data).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
print(anova_table)